In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../..")

In [2]:
import pickle
from models.bilstm_crf_model_keras import BilstmCRFKerasNerModel
from models.text_encode import SelfVocabEncoder
from models.data_parse import ConllParser
import random

In [3]:
parser = ConllParser()
train_parsed_data = parser.parse_data('data/CCKS2021中文NLP地址要素解析/train_human.conll')
test_parsed_data = parser.parse_data('data/CCKS2021中文NLP地址要素解析/dev.conll')

random.shuffle(train_parsed_data)
cutoff = int(len(train_parsed_data) * 0.8)
val_parsed_data = train_parsed_data[cutoff:]
train_parsed_data = train_parsed_data[0:cutoff]

In [4]:
model_pth = "data/CCKS2021中文NLP地址要素解析/saved_models/bilstm_crf_1.h5"
with open('./data/CCKS2021中文NLP地址要素解析/tags.pkl', 'rb') as f:
    chunk_tags = pickle.load(f)
encoder = SelfVocabEncoder()
encoder.construct_vocab(train_parsed_data)
ner = BilstmCRFKerasNerModel(chunk_tags, vocab_size=encoder.vocab_size, model_path=model_pth)
ner.encoder = encoder
# processor = ConllProcessor()
# ner.processor = processor

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         212700    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 100)         60400     
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
crf_layer (MyCRF)            (None, None)              9120      
Total params: 342,620
Trainable params: 342,620
Non-trainable params: 0
_________________________________________________________________


/Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


In [5]:
train_data = ner.preprocess(train_parsed_data)
val_data = ner.preprocess(val_parsed_data)
test_data = ner.preprocess(test_parsed_data)

y_chunk shape: (14569, 36)
y_chunk shape: (3643, 36)
y_chunk shape: (1970, 36)


In [6]:
ner.fit(train_data=train_data, val_data=val_data)

Epoch 1/10


/Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow_addons/text/crf.py:545: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  warnings.warn(


456/456 [==============================] - 207s 454ms/step - loss: 15.7554 - crf_accuracy: 0.5685 - val_loss: 5.2234 - val_crf_accuracy: 0.8435
Epoch 2/10
456/456 [==============================] - 211s 462ms/step - loss: 3.6783 - crf_accuracy: 0.8825 - val_loss: 2.4720 - val_crf_accuracy: 0.9088
Epoch 3/10
456/456 [==============================] - 216s 473ms/step - loss: 2.1646 - crf_accuracy: 0.9171 - val_loss: 1.7751 - val_crf_accuracy: 0.9228
Epoch 4/10
456/456 [==============================] - 2878s 6s/step - loss: 1.5795 - crf_accuracy: 0.9313 - val_loss: 1.4315 - val_crf_accuracy: 0.9311
Epoch 5/10
456/456 [==============================] - 203s 444ms/step - loss: 1.2466 - crf_accuracy: 0.9393 - val_loss: 1.2162 - val_crf_accuracy: 0.9366
Epoch 6/10
456/456 [==============================] - 203s 444ms/step - loss: 1.0233 - crf_accuracy: 0.9455 - val_loss: 1.0727 - val_crf_accuracy: 0.9401
Epoch 7/10
456/456 [==============================] - 203s 445ms/step - loss: 0.8647 - c